In [12]:
#Черепов Арсений Би-17-2
#ссылка на репозиторий GitHub
#https://github.com/arseniycherepov/MachineLearning
#Подключаем библиотеки
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

In [13]:
#Считываем данные с файлов
transactions = pd.read_csv('casc-resto.csv', sep=';', parse_dates = ['RKDate'], decimal=',')
clients = pd.read_csv('CASC_Constant.csv', sep =',')

In [14]:
#Выполняем очистку и подготовку данных к анализу
transactions.dropna()
transactions.drop_duplicates(keep=False, inplace=True)
clients.dropna()
clients.drop_duplicates(keep=False, inplace=True)
clients['Sex'] = clients['Sex'].astype('category').cat.codes
clients['Age'] = clients['Age'].apply(
    lambda x: int(clients['Age'].mean()) if x > 100 else x)

In [15]:
#Присваиваем стартовую и конечную дату переменным
date1 = datetime(2017, 7, 1)
date2 = datetime(2017, 12, 31)

In [16]:
#Для каждого клиента определим, совершал ли он визит в период между 2017-07-01 и 2017-12-31 включительно
def GetY(clients):
    return len(clients.loc[(clients['RKDate'] >= date1) & (clients['RKDate'] <= date2)]) > 0

#Расчитаем количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01.
def Recency(clients):    
    OrdersBefore = clients.loc[clients['RKDate'] < date1]
    LastOrder = OrdersBefore['RKDate'].max()
    return pd.Timedelta(date1 - LastOrder).days

#Определим среднюю частоту походов клиентов на дату 2017-07-01.
def Frequency(clients):
    OrdersBefore = clients.loc[clients['RKDate'] < date1]
    return OrdersBefore.groupby([OrdersBefore['RKDate'].dt.year,
                                 OrdersBefore['RKDate'].dt.month]).size().mean()

#Рассчитаем средний чек клиента по всем покупкам до 2017-07-01.
def MonetaryValue(clients):
    OrdersBefore = clients.loc[clients['RKDate'] < date1]
    return OrdersBefore['SummAfterPointsUsage'].mean()

#Расчитаем функции  
def RunFunctions(clients):
    result = [{
        'Y': GetY(clients),
        'Recency': Recency(clients),
        'Frequency': Frequency(clients),
        'MonetaryValue': MonetaryValue(clients)
    }]
    return pd.DataFrame(data = result)


In [17]:
#Вызовем итоговую функцию и притянем данные из второй таблицы
ClientResult = transactions.groupby(by='CustomerID').apply(lambda temp: RunFunctions(temp))
ClientResult = ClientResult.join(clients[['CustomerId', 'Age', 'Sex']].set_index('CustomerId'), on = ['CustomerID'], how='right')
ClientResult.dropna()

,CustomerID,Y,Recency,Frequency,MonetaryValue,Age,Sex
"(2728046, 0)",2728046,False,160.0,5.833333,272.628571,24.0,0
"(2728088, 0)",2728088,False,720.0,9.000000,288.777778,46.0,1
"(2728089, 0)",2728089,True,15.0,3.750000,404.982000,27.0,0
"(2728095, 0)",2728095,False,177.0,5.909091,304.245231,54.0,0
"(2728107, 0)",2728107,True,115.0,5.666667,323.411765,48.0,1
...,...,...,...,...,...,...,...
"(2913103, 0)",2913103,False,401.0,5.800000,562.000000,32.0,-1
"(2913106, 0)",2913106,False,126.0,7.863636,266.225202,23.0,1
"(2913114, 0)",2913114,False,95.0,7.916667,318.592632,27.0,0
"(2913123, 0)",2913123,True,8.0,10.428571,138.876712,30.0,0


In [19]:
#Выделим зависимую переменную
y = ClientResult['Y']
X = ClientResult.drop('Y', axis=1)
#Разделим данные на обучающую и тестовую выборку в соотношении 80 на 20
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25, test_size=0.2)
#Обучим модель логистической регрессии на обучающей выборке.
Regression = LogisticRegression(random_state = 25, solver='lbfgs', max_iter=120)
Regression.fit(X_train, y_train)
#Предскажем вероятность прихода на тестовой выборке.
Prediction = Regression.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#Посчитаем метрики Precision и Recall на основании предсказаний на тестовой выборке и истинных значений зависимой переменной Y
print(classification_report(y_test, y_predict))